<a href="https://colab.research.google.com/github/ambilkar26/1.-cognitive-customer-insights-with-watson-ai---7c9a3c8d/blob/main/CognitiveInsights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit pyngrok datasets ibm-watson pandas matplotlib seaborn

!pip install --upgrade datasets fsspec huggingface_hub

app_code = '''
import streamlit as st
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datasets import load_dataset
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 import Features, SentimentOptions, EmotionOptions

# Watson credentials
api_key = "NYUTTo6Sxiho7IfsU7cYGZZBt05tQGU_rSuMDPD8bSPs"
url = "https://api.au-syd.natural-language-understanding.watson.cloud.ibm.com/instances/37adc6fd-80a6-4794-9577-ad2cfaf0977f"

# Initialize Watson NLU
authenticator = IAMAuthenticator(api_key)
nlu = NaturalLanguageUnderstandingV1(
    version="2021-08-01",
    authenticator=authenticator
)
nlu.set_service_url(url)

def analyze_feedback(text):
    try:
        response = nlu.analyze(
            text=text,
            features=Features(sentiment=SentimentOptions(), emotion=EmotionOptions())
        ).get_result()
        sentiment = response["sentiment"]["document"]["label"]
        emotions = response["emotion"]["document"]["emotion"]
        return sentiment, emotions
    except:
        return "error", {}

# Load dataset directly from Hugging Face (no cache)
def load_feedback_data():
    dataset = load_dataset("sst2", split="train[:30]")  # Load only 30 for demo
    df = pd.DataFrame(dataset)
    df = df.rename(columns={"sentence": "Feedback"})
    return df[["Feedback"]]

st.set_page_config(page_title="Cognitive Customer Insights", layout="wide")
st.title("🧠 Cognitive Customer Insights")
st.write("Live sentiment and emotion analysis using IBM Watson NLU.")

# Load data
df = load_feedback_data()
st.subheader("Sample Customer Feedback")
st.dataframe(df)

st.info("Analyzing feedback... (Please wait)")
df["Sentiment"], df["Emotions"] = zip(*df["Feedback"].map(analyze_feedback))

# Show results
st.subheader("Analyzed Data")
st.dataframe(df)

# Sentiment chart
st.subheader("Sentiment Distribution")
fig1, ax1 = plt.subplots()
sns.countplot(data=df, x="Sentiment", ax=ax1)
st.pyplot(fig1)

# Emotions chart
st.subheader("Average Emotion Scores")
emotions_df = df["Emotions"].apply(pd.Series)
fig2, ax2 = plt.subplots()
emotions_df.mean().plot(kind="bar", ax=ax2)
st.pyplot(fig2)

st.download_button("Download Results", df.to_csv(index=False), "feedback_insights.csv", "text/csv")
'''
with open("app.py", "w") as f:
    f.write(app_code)


import os, threading
from pyngrok import ngrok

# Run Streamlit
def run():
    os.system("streamlit run app.py")
threading.Thread(target=run).start()

# Create tunnel
ngrok.set_auth_token("2wqR6082tM2QV5p6dyttmpf4FsI_82vqkkPCtSHSV2vzdp9oy")
public_url = ngrok.connect(8501)
print("App running at:", public_url)

  Using cached fsspec-2025.5.1-py3-none-any.whl.metadata (11 kB)
App running at: NgrokTunnel: "https://74a0-34-16-194-221.ngrok-free.app" -> "http://localhost:8501"
